In [7]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from os import listdir
from os.path import isfile, join

import PIL
from PIL import Image

import requests
import io

In [19]:
url = "https://raw.githubusercontent.com/JulienJ-44/rakuteam/main/Datasets/X_test_update.csv" 
download = requests.get(url).content

X_test= pd.read_csv(io.StringIO(download.decode('utf-8')))

X_test.head()

,Unnamed: 0,designation,description,productid,imageid
0,84916,Folkmanis Puppets - 2732 - Marionnette Et Théâ...,NaN,516376098,1019294171
1,84917,Porte Flamme Gaxix - Flamebringer Gaxix - 136/...,NaN,133389013,1274228667
2,84918,Pompe de filtration Speck Badu 95,NaN,4128438366,1295960357
3,84919,Robot de piscine électrique,<p>Ce robot de piscine d&#39;un design innovan...,3929899732,1265224052
4,84920,Hsm Destructeur Securio C16 Coupe Crois¿E: 4 X...,NaN,152993898,940543690


In [20]:
url = "https://raw.githubusercontent.com/JulienJ-44/rakuteam/main/Features/df_features_images.csv" 
download = requests.get(url).content

features_train = pd.read_csv(io.StringIO(download.decode('utf-8')))

features_train.head()

,prdtypecode,productid,blanc,noir,R,G,B
0,2583,3936362802,0.459121,0.005839,191.914832,205.177432,211.476028
1,2583,3147314797,0.824865,0.005411,230.272176,231.747476,229.365300
2,2583,207151480,0.905972,0.003960,244.311256,244.458700,243.333984
3,2583,4128438373,0.624599,0.001893,186.745312,187.626912,187.789808
4,2583,1857269337,0.777468,0.001247,216.314272,216.855752,215.883308


In [24]:
X_test = X_test.drop(["Unnamed: 0", "designation", "description"], axis=1)
X_test.head()

,productid,imageid
0,516376098,1019294171
1,133389013,1274228667
2,4128438366,1295960357
3,3929899732,1265224052
4,152993898,940543690


In [28]:
X_test["image_name"] = "image_" + X_test.imageid.map(str)+ "_product_" + X_test.productid.map(str) + ".jpg"
X_test["image_name"] = X_test["image_name"].astype(str)

#changer le path au path de vos images
X_test['fullpath']= '/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/images/image_test/'+ X_test['image_name']
X_test.head()

,productid,imageid,image_name,fullpath
0,516376098,1019294171,image_1019294171_product_516376098.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
1,133389013,1274228667,image_1274228667_product_133389013.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
2,4128438366,1295960357,image_1295960357_product_4128438366.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
3,3929899732,1265224052,image_1265224052_product_3929899732.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...
4,152993898,940543690,image_940543690_product_152993898.jpg,/Users/ayseaylinkaya/Desktop/Datascientist/2 P...


In [29]:
X_test.shape

(13812, 4)

In [31]:
images_test = np.array([np.array(Image.open(fname)) for fname in X_test['fullpath']])

blanc = []
noir = []
R = []
G = []
B = []

for image in images_test:
    n_white_pix = np.sum(image == [255, 255, 255])/750000
    n_black_pix = np.sum(image == [0,0,0])/750000
    image_mean = np.mean(image, axis=(0, 1))
    blanc.append(n_white_pix)
    noir.append(n_black_pix)
    R.append(image_mean[0])
    G.append(image_mean[1])
    B.append(image_mean[2])

X_test['blanc']=blanc
X_test['noir']=noir
X_test['R']= R
X_test['G']= G
X_test['B']= B

features_test = X_test.drop(["image_name", "fullpath"], axis=1)
features_test.head()                           

,productid,imageid,blanc,noir,R,G,B
0,516376098,1019294171,0.388052,0.000000,238.219716,231.545348,224.218576
1,133389013,1274228667,0.270684,0.001523,160.739176,141.294272,137.946564
2,4128438366,1295960357,0.624599,0.001893,186.745312,187.626912,187.789808
3,3929899732,1265224052,0.484449,0.003077,189.173824,179.905552,194.486040
4,152993898,940543690,0.834361,0.000269,242.554956,242.879876,243.262268


In [32]:
features_test.shape #contrôle de shape

(13812, 7)

In [35]:
path = '/Users/ayseaylinkaya/Desktop/Datascientist/2 Project Rakuten/'
features_test.to_csv(f'{path}/features_test.csv', index=False) #création de csv features images pour les images TEST

In [36]:
features_images_all = pd.concat([features_train, features_test])
features_images_all.to_csv(f'{path}/features_images_all.csv', index=False) #création de csv features images pour les images TRAIN + TEST
features_images_all.shape

(98728, 8)